In [2]:
%pip install fuzzytm
%pip install gensim

  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached scipy-1.10.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (53 kB)
Using cached scipy-1.10.1-cp39-cp39-macosx_12_0_arm64.whl (28.9 MB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
from FuzzyTM import FLSA_W, FLSA

In [6]:
# import the data
path = '../Assignment-1/Dataset/assignment-2-data.csv'
df_cleaned = pd.read_csv(path)

df_cleaned['new_tokenized'] = df_cleaned['clean_content'].apply(lambda x: x.split())

df_cleaned.head(5)

,id,ticker,title,category,content,release_date,provider,url,article_id,Date,Open,Close,clean_content,label,tokenized_content,new_tokenized
0,221539,NIO,A Central Bank War Just Started And Its Good F...,opinion,ECB Effects\nThe move in the euro was huge fa...,2019-03-07,Michael Kramer,https://www.investing.com/analysis/a-central-b...,200395687,2019-03-07,3.77750,3.73150,ecb effect move euro huge falling pip huge imp...,0,"['ecb', 'effect', 'move', 'euro', 'huge', 'fal...","[ecb, effect, move, euro, huge, falling, pip, ..."
1,221547,NIO,6 Stocks To Watch Nivida Could Be Falling,opinion,6 Stocks To Watch March 6 Trading Session\nSt...,2019-03-06,Michael Kramer,https://www.investing.com/analysis/6-stocks-to...,200394931,2019-03-06,3.90400,3.80125,stock watch march trading session stock stock ...,0,"['stock', 'watch', 'march', 'trading', 'sessio...","[stock, watch, march, trading, session, stock,..."
2,221572,NIO,Stocks Dow Drops Nearly 400 Points as Apple ...,news,Investing com A rout in Apple and Facebook ...,2018-11-19,Investing.com,https://www.investing.com/news/stock-market-ne...,1694042,2018-11-19,4.04475,3.61750,investing com rout apple facebook nasdaq fb mo...,0,"['investing', 'com', 'rout', 'apple', 'faceboo...","[investing, com, rout, apple, facebook, nasdaq..."
3,221597,UBER,The Best Of CES 2020 Revised,opinion,With 4 500 companies bringing their innovation...,2020-01-16,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200499164,2020-01-16,6.19300,6.22325,company bringing innovation ce jan get really ...,1,"['company', 'bringing', 'innovation', 'ce', 'j...","[company, bringing, innovation, ce, jan, get, ..."
4,221614,UBER,The Best Of CES 2020,opinion,With 4 500 companies bringing their innovation...,2020-01-10,Zacks Investment Research,https://www.investing.com/analysis/the-best-of...,200498063,2020-01-10,6.18325,6.10800,company bringing innovation ce jan get really ...,0,"['company', 'bringing', 'innovation', 'ce', 'j...","[company, bringing, innovation, ce, jan, get, ..."


In [7]:
input_file = df_cleaned['new_tokenized'].to_list()

In [8]:
flsaW = FLSA(input_file=input_file, num_topics=10, num_words=10)

print(flsaW.get_vocabulary_size())

pwgt,ptgd = flsaW.get_matrices()

flsaW.show_topics()

for topic in flsaW.show_topics(representation='words'):
  print(topic)

print(flsaW.get_coherence_score())
print(flsaW.get_diversity_score())
print(flsaW.get_interpretability_score())

29908


TypeError: type matrix doesn't define __round__ method

In [30]:
# split the data into train, validation and test
train = input_file[:int(len(input_file)*0.8)]
test = input_file[int(len(input_file)*0.8):]

In [36]:
best_score = 0
best_params = {}

grid = {
    'num_topics': [5, 10, 30, 50, 75, 100],
    'num_words': [5, 10, 30, 50, 75, 100]
}

# Iterate over the parameters
for num_topics in grid['num_topics']:
    for num_words in grid['num_words']:
        print(f"Training FLSA model with num_topics={num_topics}, num_words={num_words}")
        
        flsaW = FLSA_W(input_file=train, num_topics=num_topics, num_words=num_words)
        
        print(f"Vocabulary size: {flsaW.get_vocabulary_size()}")
        
        pwgt, ptgd = flsaW.get_matrices()
        
        print(f"Topics: {flsaW.show_topics()}")
        
        coherence_score = flsaW.get_coherence_score()
        diversity_score = flsaW.get_diversity_score()
        interpretability_score = flsaW.get_interpretability_score()
        
        print(f"Coherence Score: {coherence_score}")
        print(f"Diversity Score: {diversity_score}")
        print(f"Interpretability Score: {interpretability_score}")
        
        if (coherence_score + diversity_score + interpretability_score > best_score):
            best_params = {'num_topics': num_topics, 'num_words': num_words}
            best_score = coherence_score + diversity_score + interpretability_score
        
# Train the best model on the combined train and validation data
test_flsa_model = FLSA_W(input_file=test, num_topics=best_params['num_topics'], num_words=best_params['num_words'])

test_coherence_score = test_flsa_model.get_coherence_score()
test_diversity_score = test_flsa_model.get_diversity_score()
test_interpretability_score = test_flsa_model.get_interpretability_score()


print(f"Test Coherence Score: {test_coherence_score}")
print(f"Test Diversity Score: {test_diversity_score}")
print(f"Test Interpretability Score: {test_interpretability_score}")

Training FLSA model with num_topics=5, num_words=5
Vocabulary size: 28219


AttributeError: 'matrix' object has no attribute 'multiply'